## Introduction to Importance Sampling

A large number of Monte Carlo methods exist for generating samples or estimating quantities of interest from a chosen distribution. Such methods are usually used when these distributions are complex and standard analytical methods for accomplishing these goals are unavailable. Monte Carlo methods are defined by the use of pseudo-random numbers, hence their nominal connection to the famous [Monte Carlo casino](https://en.wikipedia.org/wiki/Monte_Carlo_Casino).

Here, we briefly explore one technique called importance sampling. This will give you some sense for how Monte Carlo methods work generally, but also why this fairly simple approach is not efficient for complex problems, such as arise in phylogenetics. As we discuss this and other Monte Carlo approaches, we will leave formal proofs of their accuracy and more verbose descriptions of the relevant background to the textbooks. Instead, we will focus on providing an intuition for how they work.

To begin, we're just going to import some modules and classes that we'll use later to calculate probabilities and create plots.

In [ ]:
# Importing the binomial and uniform classes from the stats module of the scipy library
from scipy.stats import binom, uniform

# Importing pseudo-random number generators for uniform and Gaussian distributions
from random import random, gauss

As we did with the Introduction to Likelihood, we're going to use a coin-flipping binomial example. Bear in mind, however, that all of the estimation machinery is general to any sort of distribution in which we may be interested. In fact, feel free to make a copy of this notebook, then substitute a different type of data. You'll just need to change the function calls for calculating probabilities and the range of parameter values you're considering. I will attempt to highlight the areas that would need to be changed with comments in the code.

Here, we're starting with 5 coin flips and using the same set of observations that we explored with likelihood, in order to be able to compare results.

In [ ]:
# Defining the data
flips = ["H","T","T","T","H"]
# flips = flips*100  # Uncomment (and modify) this line to create a more informative dataset
n = len(flips)        # Number (n) of binomial trials
k = sum([1 for fl in flips if fl == "H"])  # Counting heads as successes (k)

In this example, we're going to be sampling from the posterior distribution for *p*, the binomial parameter. Below we define three functions to calculate the likelihood, the prior, and the posterior. Since *p* only has meaning on the interval [0,1], the likelihood function will return 0 for any values outside that range. Similarly, we are using a uniform prior for [0,1] - this has a constant density of 1 in that range and 0, otherwise. Even though the function to calculate prior density only has a single line, defining such a function allows us to use prior() throughout the code and only change the one line in the function if we decide to change our prior specification. The unnormalized posterior density is calculated simply by multiplying the likelihood and the prior.

In [ ]:
# Defining general function to calculate likelihoods if p<0 or p>1
def like(successes,trials,prob,testingPrior=False):
    if testingPrior:  # If True, this will always return 1. This can be useful if one wants
        return 1      # to test the machinery by estimating a known distribution (the prior).
    if prob < 0:
        return 0
    elif prob > 1:
        return 0
    else:
        return binom.pmf(successes,trials,prob)
    
# Defining function to calculate prior density - uniform [0,1]
def prior(prob):
    return uniform.pdf(prob)

# Defining function to calculate the unnormalized posterior density
def posterior(successes,trials,prob):
    posterior = prior(prob) * like(successes,trials,prob)
    return posterior

Here's the general idea of importance sampling. We don't know how to sample directly from some distribution we're interested in (we'll call it *p*) and/or we don't know how to calculate a relevant quantity (like the mean). We may not even be able to directly calculate probability densities of *p*. However, we *do* know how to sample from another distribution (say, *q*) that covers the same range of values as *p* and we can at least calculate an unnormalized density that is proportional to the pdf of *p*. If so, we can make *n* draws from the sampling distribution (*q*), then calculate weights (e.g., $w_1, w_2,\ldots,w_n$) for these samples (e.g., $x_1,x_2,\ldots,x_n$) to adjust for the fact that they were drawn from *q* and not *p*. These weights correspond to the ratio of the two probability densities (e.g., $\frac{p(x_1)}{q(x_1)}$). To calculate values of interest for a random variable *X*, distributed according to *p* (i.e., $X \sim p$), we can use the values we've drawn from *q* adjusted by their weights. For instance, $E[X]\ =\ x_1w_1+x_2w_2+\ldots+x_nw_n\ =\ x_1\frac{p(x_1)}{q(x_1)}+x_2\frac{p(x_2)}{q(x_2)}+\ldots+x_n\frac{p(x_n)}{q(x_n)}$.

In [ ]:
# To get a sense for how well importance sampling is working, we're going to run our
# experiment several times. This list will hold the estimates of the means for all 
# replicates.
estimates = []

# The number of replicates we will run.
numReps = 100

# This value establishes the upper end of the uniform distribution from which we will sample
# parameter values. Since only values between 0 and 1 can have likelihoods > 0, the more we
# extend this value above 1, the greater the disparity between our sampling distribution and
# distribution of interest. What happens as this gets bigger?
uniScale = 1

# Initializing our ad hoc progress bar
print("Progress (each . = 10 replicates): ",end="")

# Iterating across our replicates 0,...,numReps-1
for rep in range(numReps):
    
    # Incrementing our progress
    if rep % 10 == 0:
        print(".",end="")
    
    # Draw values from uniform prior using the uniform class we imported from scipy
    numValues = 100
    p_vals = uniform.rvs(size=numValues,loc=0,scale=uniScale)
    
    # Calculate initial weights (not necessarily normalized)
    weights = [(posterior(k,n,param)/uniform.pdf(param,loc=0,scale=uniScale)) for param in p_vals]
    
    # Normalize weights so average is 1
    """NOTE: This normalization isn't strictly necessary if both functions used to calculate 
           the initial weights are proper probability density functions. But even then, it 
           helps with rounding error."""
    weights = [w/np.mean(weights) for w in weights]
        
    # Calculating weighted average
    count = 0
    estMean = 0
    while (count < (len(p_vals))):
        estMean += p_vals[count]*weights[count]  # Multiplying each value by its weight
        count += 1
    estMean /= numValues
    estimates.append(estMean)
    
# Printing out some useful summary information
print()
print("The last estimated mean (replicate %d): %f." % (numReps,estimates[numReps-1]))
print("The mean of the estimated means across all %d replicates: %f." % (numReps,np.mean(estimates)))
print("The standard deviation of the estimated means is: %f."% (np.std(estimates)))


# Rerun this several times, varying the size of the dataset, the upper end of the sampling
# distribution, and the number of values drawn (numValues) from the sampling distribution. 
# Pay attention to whether/how several things change:
# - Is an error thrown?
# - The estimated means
# - The standard deviation of the estimated means
# - The run time

# Now try changing the testingPrior argument to True for the likelihood function.
# What happens? Why might this be useful?